In [ ]:
import tensorflow as tf
import os
import json

In [ ]:
os.environ['TF_CONFIG'] = json.dumps(
{
  "cluster": {
    "coordinator": ["192.168.2.1:2223"],
    "ps": ["192.168.2.2:2223"],
    "worker": ["192.168.2.3:2223"]
  },
  "task": {
    "type": "coordinator",
    "index": 0
  }
})
os.environ["grpc_fail_fast"]="use_caller"

# Create a TensorFlow ClusterSpec
cluster_spec = tf.train.ClusterSpec({
    "coordinator": ["192.168.2.1:2223"],
    "ps": ["192.168.2.2:2223"],
    "worker": ["192.168.2.3:2223"]
})

# Start a TensorFlow server
resolver = tf.distribute.cluster_resolver.SimpleClusterResolver(cluster_spec=cluster_spec)
server = tf.distribute.Server(cluster_spec, job_name="coordinator", task_index=0)

In [ ]:
strategy = tf.distribute.ParameterServerStrategy(resolver)

In [ ]:
(x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10).repeat()
dataset = dataset.batch(10)
dataset = dataset.prefetch(2)
print("checkpoint 2")

In [ ]:
with strategy.scope():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
        ])  # Make sure the `Model` is created within scope.
    
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", steps_per_execution=10)

In [ ]:
model.fit(dataset, epochs=20, steps_per_epoch=15)